# Forward UQ with QMCpy and UM-Bridge

* Goal: Perform forward UQ with uncertain parameter $\theta \sim \mathcal{U}([3.4, 3.5] \times [1.0, 1.2])$
* Connect to a running model via UM-Bridge, providing $F : \mathbb{R}^n \rightarrow \mathbb{R}^m$.
* Compute $\mathbb{E}[F(\theta)]$ via quasi-Monte Carlo, using QMCpy package

In [ ]:
import qmcpy as qp
from qmcpy.integrand.um_bridge_wrapper import UMBridgeWrapper
import umbridge

# Set up UM-Bridge model
model = umbridge.HTTPModel("http://localhost:4242", "forward")

d = 2

# Choose a distribution of suitable dimension to sample via QMC
dnb2 = qp.DigitalNetB2(d)

gauss_sobol = qp.Uniform(dnb2, lower_bound=[3.4, 1.0], upper_bound=[3.5, 1.2])

# Create integrand based on umbridge model
integrand = UMBridgeWrapper(gauss_sobol, model, parallel=False)

# Run QMC integration to some accuracy and print results
qmc_sobol_algorithm = qp.CubQMCSobolG(integrand, abs_tol=1e10, n_init = 2**8, n_max = 2**8)
solution,data = qmc_sobol_algorithm.integrate()
print(data)

In [ ]:
import matplotlib.pyplot as plt

for row in data.yfull:
  plt.plot(row[0:201], row[201:402], alpha=0.2)
plt.show()

## Probability of impact
The model returns
* 201 trajectory entries for x values
* 201 trajectory entries for y values
* 1 binary entry indicating impact on moon
* 1 binary entry indicating impact on earth
* 2 entries for position of spacecraft at t=1e4
* 2 entries for position of spacecraft at t=2e4

Note: QMCpy flattens the output into a single array!

In [ ]:
# Currently prints the expected value of impact on earth
# What is the expected value of impact on moon?
print(solution[403])